In [1]:
import os
from torch_geometric.data import Data
import torch

In [64]:
dataset = []

for ptfile in os.listdir('concept_drift_101'):
    dataset.append(torch.load(f'concept_drift_101/{ptfile}'))

In [56]:
sum=0
for i in dataset:
    sum += len(i.y)

In [4]:
sum

3213

In [65]:
train_data = dataset[0:int(0.8*366)]
test_data = dataset[int(0.8*366):]

In [6]:
dataset[1].edge_to_edge_adj_matr.count_nonzero()

tensor(11)

In [7]:
dataset[1].y

tensor([0, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0])

In [8]:
dataset[1].edge_attr

tensor([[0.3293, 0.0099, 0.1074, 0.0114, 0.2529, 0.0128, 0.8892, 0.0058, 0.0072,
         0.0142, 0.8723],
        [0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767, 0.6767,
         0.6767, 0.6767],
        [1.0000, 0.0000, 0.0000, 1.0000, 1.0000, 0.0000, 1.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 1.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 1.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 0.0000],
        [0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 1.0000, 0.0000, 0.0000, 0.0000,
         1.0000, 0.0000],
        [0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000, 0.0000,
         0.0000, 1.0000]], dtype=torch.float64)

In [66]:
len(train_data)

292

In [10]:
len(test_data)

74

In [11]:
from model_nenn import Nenn

In [12]:
dataset[0].x.shape

torch.Size([6, 18])

In [13]:
dataset[0].edge_attr.shape

torch.Size([8, 9])

In [22]:
model = Nenn(6,8,6,8,2)

In [23]:
model = model.to('cpu')

In [24]:
loss = torch.nn.CrossEntropyLoss()
# TODO Undersampling Smote
# TODO Oversampling
optimizer = torch.optim.Adam(model.parameters(), lr=0.0001)

In [25]:
dataset[0]

Data(x=[6, 18], edge_index=[2, 9], edge_attr=[8, 9], y=[9], node_to_node_adj_matr=[18, 18], node_to_edge_adj_matr=[18, 9], edge_to_node_adj_matr=[9, 18], edge_to_edge_adj_matr=[9, 9])

In [26]:
def train():
    model.train()
    # Enumerar sobre os dados.
    for epoch in range(100):
        for ts, data in enumerate(train_data):
        
            # Resetar Gradientes
            optimizer.zero_grad()
            # Passando as informações do batch e de conexão dos grafos
            logits, _ = model(
                data.x.T.type(torch.FloatTensor),
                data.edge_attr.T.type(torch.FloatTensor),
                data.edge_to_edge_adj_matr.T.type(torch.FloatTensor),
                data.edge_to_node_adj_matr.T.type(torch.FloatTensor),
                data.node_to_edge_adj_matr.T.type(torch.FloatTensor),
                data.node_to_node_adj_matr.T.type(torch.FloatTensor)
            )
            #label_pred = pred.max(1)[1]
            # Calculando a perda e os gradientes.
            l = loss(logits, data.y)
            l.backward()
            # Atualizar usando os gradientes.
            optimizer.step()
        print('epoch =', epoch + 1, 'loss =', l.item())

In [27]:
train()

epoch = 1 loss = 0.6576309204101562
epoch = 2 loss = 0.6456303000450134
epoch = 3 loss = 0.6346338391304016
epoch = 4 loss = 0.6251981854438782
epoch = 5 loss = 0.617359459400177
epoch = 6 loss = 0.611345648765564
epoch = 7 loss = 0.6069813966751099
epoch = 8 loss = 0.6039942502975464
epoch = 9 loss = 0.602070689201355
epoch = 10 loss = 0.6009279489517212
epoch = 11 loss = 0.6003531813621521
epoch = 12 loss = 0.6002811193466187
epoch = 13 loss = 0.6005963087081909
epoch = 14 loss = 0.6012243032455444
epoch = 15 loss = 0.6021098494529724
epoch = 16 loss = 0.6031619310379028
epoch = 17 loss = 0.6043581962585449
epoch = 18 loss = 0.60566645860672
epoch = 19 loss = 0.6070435643196106
epoch = 20 loss = 0.6084668636322021
epoch = 21 loss = 0.609906792640686
epoch = 22 loss = 0.6113606095314026
epoch = 23 loss = 0.6129534840583801
epoch = 24 loss = 0.6146257519721985
epoch = 25 loss = 0.6161980032920837
epoch = 26 loss = 0.617746114730835
epoch = 27 loss = 0.619293749332428
epoch = 28 loss = 

In [67]:
label_pred_list = []
y_true_list = []

def test():
    model.eval()
    with torch.no_grad():
        global label_pred_list
        global y_true_list
        for data in test_data:
            data.to('cpu')
            logits, _ = model(
                data.x.T.type(torch.FloatTensor),
                data.edge_attr.T.type(torch.FloatTensor),
                data.edge_to_edge_adj_matr.T.type(torch.FloatTensor),
                data.edge_to_node_adj_matr.T.type(torch.FloatTensor),
                data.node_to_edge_adj_matr.T.type(torch.FloatTensor),
                data.node_to_node_adj_matr.T.type(torch.FloatTensor)
            )
            label_pred = logits.max(1)[1].tolist()
            label_pred_list += label_pred
            y_true_list += data.y.tolist()
    model.train()

In [68]:
test()

In [69]:
import numpy as np
np.sum(label_pred_list)

316

In [70]:
label_pred_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,


In [50]:
y_true_list

[0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 1,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 1,
 1,
 1,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,
 0,


In [51]:
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report

In [71]:
mean_precision = []
mean_recall = []
mean_f1 = []

prec = precision_score(y_true_list, label_pred_list, average='binary', labels=[0])
rec = recall_score(y_true_list, label_pred_list, average='binary', labels=[0])
f1 = f1_score(y_true_list, label_pred_list, average='binary', labels=[0])

In [72]:
print(classification_report(y_true_list,label_pred_list))

              precision    recall  f1-score   support

           0       0.99      0.93      0.96      1923
           1       0.54      0.90      0.68       192

    accuracy                           0.92      2115
   macro avg       0.77      0.91      0.82      2115
weighted avg       0.95      0.92      0.93      2115



In [36]:
prec

0.8861386138613861

In [37]:
rec

0.9322916666666666

In [38]:
f1

0.9086294416243654

In [39]:
prec = precision_score(y_true_list, label_pred_list, average='macro')
rec = recall_score(y_true_list, label_pred_list, average='macro')
f1 = f1_score(y_true_list, label_pred_list, average='macro')

In [40]:
prec

0.9286568900792518

In [41]:
rec

0.9412000632682573

In [42]:
f1

0.9345778787069196